# 0. 準備

## 0-1. ライブラリのインストール&インポート

In [ ]:
import os
import sys

# レポジトリをクローンして移動
REPO_DPATH = 'clip-prefix-caption-jp'
if not os.path.exists(REPO_DPATH):
  !git clone https://github.com/ohashi56225/clip-prefix-caption-jp.git
sys.path.append(REPO_DPATH)
%cd $REPO_DPATH

# 必要ライブラリインストール
!pip install git+https://github.com/openai/CLIP.git
!pip install scikit-image torch transformers sentencepiece

# インポート
import json
import random
import gdown
from test import Predictor
from model import build_model
from IPython.display import display
from google.colab import files

# ついでに後で使う関数も作っておく
def upload_file():
  uploaded = files.upload()
  if not uploaded:
    image_fpath = ''
  elif len(uploaded) == 1:
    image_fpath = list(uploaded.keys())[0]
  else:
    raise RuntimeError("1度に1枚まで")
  return image_fpath

## 0-2. データダウンロード

In [ ]:
# 画像データ
gdown.download("https://drive.google.com/uc?id=18j0Cx5aPfuBkCfD0RlWGYpYSs66P6Frq", "data.zip", quiet=False)
!unzip data.zip -d data

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--id] url_or_id
gdown: error: unrecognized arguments: data model.py notebooks __pycache__ README.md test.py train.py
unzip:  cannot find or open checkpoints.zip, checkpoints.zip.zip or checkpoints.zip.ZIP.
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


## 1. SFCOCOデータセットのみで実験

## 1-1. 学習

In [ ]:
!python train.py --model_name sfcoco \
                 --train_data_fpath data/sfcoco/processed/train.pkl \
                 --valid_data_fpath data/sfcoco/processed/valid.pkl \
                 --epochs 20 \
                 --batch_size 4

## 1-2. 学習後モデル読み込み

In [ ]:
sfcoco_model = build_model(model_fpath="")
sfcoco_predictor = Predictor(model=sfcoco_model)

## 1-3. テスト画像の読み込み

In [ ]:
#@markdown ### 好きな画像をアップロードする場合
#@markdown ローカルにある画像を使いたい場合は，このセルを実行してアップロードしてください
#@markdown アップロードした画像はカレントディレクトリ直下に吐き出されます

image_fpath = upload_file()

In [ ]:
#@markdown ### テスト画像リストから選ぶ場合
#@markdown テスト画像リストの画像を使用する場合は，このセルを実行して1枚選択してください．

# テスト画像ファイルリスト読込
TEST_IMAGE_FNAME_LIST = json.load(open("data/sfcoco/processed/test_list.json"))

# 1枚選択
image_fname = TEST_IMAGE_FNAME_LIST[1] # [1]は福沢諭吉像の画像
# image_fname = random.choice(TEST_IMAGE_FNAME_LIST)

image_fpath = os.path.join("data/sfcoco/images", image_fname)

## 1-4. キャプション生成

In [ ]:
pil_image, captions = sfcoco_predictor.caption(image_fpath=image_fpath, beam_size=5)
display(pil_image) # 画像を表示
print(json.dumps(captions, indent=2, ensure_ascii=False)) # キャプションを表示

In [ ]:
for image_fname in TEST_IMAGE_FNAME_LIST[:2]:
    image_fpath = os.path.join("data/sfcoco/images", image_fname)
    pil_image, captions = coco_predictor.caption(image_fpath=image_fpath, beam_size=5)
    print(json.dumps(captions, indent=2, ensure_ascii=False)) # キャプションを表示

# 2. COCOデータセットのみで実験

## 2-1. モデル準備

In [ ]:
# 学習済みモデルの重みをダウンロード
gdown.download("", "coco_prefix-***.zip", quiet=False)
!unzip coco_prefix-***.zip -d checkpoints

# # 学習済みモデルを読み込み
coco_model = build_model(model_fpath="checkpoints/coco-004.pt")
coco_predictor = Predictor(model=coco_model)

## 2-2. テスト画像の読み込み

In [ ]:
#@markdown ### 好きな画像をアップロードする場合
#@markdown ローカルにある画像を使いたい場合は，このセルを実行してアップロードしてください．
#@markdown アップロードした画像はカレントディレクトリ直下に吐き出されます．

image_fpath = upload_file()

In [6]:
#@markdown ### テスト画像リストから選ぶ場合
#@markdown テスト画像リストの画像を使用する場合は，このセルを実行して1枚選択してください．

# テスト画像ファイルリスト読込
TEST_IMAGE_FNAME_LIST = json.load(open("data/sfcoco/processed/test_list.json"))

# 1枚選択
image_fname = TEST_IMAGE_FNAME_LIST[1] # [1]は福沢諭吉像の画像
# image_fname = random.choice(TEST_IMAGE_FNAME_LIST)

image_fpath = os.path.join("data/sfcoco/images", image_fname)

## 2-3. キャプション生成

In [ ]:
# キャプション生成
pil_image, captions = coco_predictor.caption(image_fpath=image_fpath, beam_size=5)
# display(pil_image) # 画像を表示
print(json.dumps(captions, indent=2, ensure_ascii=False)) # キャプションを表示

# 3. COCOデータセット+SFCOCOデータセットを合わせて実験

## 3-1. モデル準備

In [ ]:
gdown.download("", "cocosfcoco-***.zip", quiet=False)
!unzip cocosfcoco-***.zip -d checkpoints

# 学習済みモデルと推論器を読み込む
cocosfcoco_model = build_model(model_fpath="checkpoints/cocosfcoco-008.pt")
cocosfcoco_predictor = Predictor(model=cocosfcoco_model)

Train both prefix and GPT
Resume pretrained weights from checkpoints/cocosfcoco_prefix-017.pt


## 3-2. テスト画像読み込み

In [ ]:
#@markdown #### 好きな画像をアップロードする場合
image_fpath = upload_file()

In [ ]:
#@markdown ### テスト画像リストから選ぶ場合
#@markdown テスト画像リストの画像を使用する場合は，このセルを実行して1枚選択してください．

# テスト画像ファイルリスト読込
TEST_IMAGE_FNAME_LIST = json.load(open("data/sfcoco/processed/test_list.json"))

# 1枚選択
image_fname = TEST_IMAGE_FNAME_LIST[1] # [1]は福沢諭吉像の画像
# image_fname = random.choice(TEST_IMAGE_FNAME_LIST)

image_fpath = os.path.join("data/sfcoco/images", image_fname)

## 3-2. キャプション生成

In [ ]:
pil_image, captions = cocosfcoco_predictor.caption(image_fpath=image_fpath, beam_size=5)
display(pil_image) # 画像を表示
print(json.dumps(captions, indent=2, ensure_ascii=False)) # キャプションを表示